In [22]:
# re
import re

from gut_tokenize import read_data, preprocess

# sentence tokenize / parse
import spacy
en = spacy.load('en_core_web_sm')
# enforce max length
en.max_length = 1500000

In [23]:
directory = "../Gutenberg/cleaned_texts/"

titles, texts = read_data(directory)

print(titles)

# germinal = preprocess(texts[0])

# germinal_clauses = clause_parse(germinal)

['robinson_crusoe_clean.txt', 'great_gatsby_clean.txt', 'around_the_world_in_80_days_clean.txt', 'germinal-clean.txt', 'pride_and_prejudice_clean.txt', 'the_jungle_clean.txt', 'scarlet_letter_clean.txt', 'great_expectations_clean.txt', 'moby-clean.txt', 'anna_karenina_clean.txt', 'twenty_thousand_leagues_under_the_sea_clean.txt', 'alice_in_wonderland_clean.txt', 'frankenstein_clean.txt', 'sense_and_sensibility_clean.txt', 'middlemarch_clean.txt', 'tom_jones_clean.txt', 'utopia_clean.txt', 'the_time_machine_clean.txt', 'north_and_south_clean.txt', 'the_fortune_of_the_rougons_clean.txt', 'adam_bede_clean.txt', 'babbit_clean.txt', 'the_last_man_clean.txt', 'madame_bovary_clean.txt', 'picture_of_dorian_grey_clean.txt', 'heart_of_darkness_clean.txt', 'crime_and_punishment_clean.txt', 'bleak_house_clean.txt']


In [5]:
'''
This method relies on the spacy parser and returns a clause-level tokenization of a given text.
Where "clause" is simply text between two delimiters like commas.
E.g. the sentence
"this is a test, of many things, but mostly your abilities." would split into:

- this is a test
- of many things
- but mostly your abilities

See https://stackoverflow.com/a/65300589 for germ of method logic

Args:
    text: document-level rep of text after being read in and roughly-cleaned
 
Returns:
    texts: cleaned. a list holding each clause for the text
'''
def clause_parse(text):
    doc = en(text)
    
    # keep track of covered words
    seen = set()

    chunks = []
    for sent in doc.sents:
        heads = [cc for cc in sent.root.children if cc.dep_ == 'conj']

        for head in heads:
            words = [ww for ww in head.subtree]
            for word in words:
                seen.add(word)
            chunk = (' '.join([ww.text for ww in words]))
            chunks.append((head.i, chunk))

        unseen = [ww for ww in sent if ww not in seen]
        chunk = ' '.join([ww.text for ww in unseen])
        chunks.append((sent.root.i, chunk))

    chunks = sorted(chunks, key=lambda x: x[0])

    cleaned = []

    for ii, chunk in chunks:
        # replace boundary char
        chunk = chunk.replace(",", "SEP")
        chunk = chunk.replace(".", "SEP")
        # also replace orphan "and" sections
        chunk = [c.strip() for c in chunk.split("SEP") if c.strip() not in ("", "and", None)]
        cleaned.append(chunk)
    
    clauses = sum(cleaned, [])
    
    return [clause for clause in clauses if clause != ""]

In [7]:
print(germinal[0: 558])

print('============')

clause_count = 0
for c in germinal_clauses:
    print (c)
    clause_count += 1
    if clause_count == 11:
        break

 over the open plain, beneath a starless sky as dark and thick as ink, a man walked alone along the highway from marchiennes to montsou, a straight paved road ten kilometres in length, intersecting the beetroot fields. he could not even see the black soil before him, and only felt the immense flat horizon by the gusts of march wind, squalls as strong as on the sea, and frozen from sweeping leagues of marsh and naked earth. no tree could be seen against the sky, and the road unrolled as straight as a pier in the midst of the blinding spray of darkness. 
over the open plain
beneath a starless sky as dark and thick as ink
a man walked alone along the highway from marchiennes to montsou
a straight paved road ten kilometres in length
intersecting the beetroot fields
he could not even see the black soil before him
only felt the immense flat horizon by the gusts of march wind
squalls as strong as on the sea
and frozen from sweeping leagues of marsh and naked earth
no tree could be seen agains

In [271]:
def nouns_and_modifiers(clauses):
    
    # obj: vanilla object
    # pobj: object of a pr
    # iobj: indirect obj
    # pobj: prep. obj
    # npadvmod: noun phrase as adverbial modifier (common for measurements; 20 years old)
    # nsubj: noun subject
    # nsubjpass: passive nominal subject
    # acomp: adjectival complement
    
    np_labels = ['compound', 'obj','dobj','iobj','pobj','npadvmod','nsubj','nsubjpass']
    
    # modifiers ... acl, amod, nummod, nn, advmod
    
    detail_dict = {}
    count = 0
    for clause in clauses:
        print("==============")
        doc = en(clause)
        for word in doc:
            # if word.dep_ in np_labels 
            if word.pos_ == "NOUN" or word.pos_ == "VERB":
                detail_chunks = [] 
                size = sum(1 for dummy in word.subtree)
                if size > 1:
                    detail_chunk = []
                    for descendant in word.subtree:
                        detail_chunk.append(descendant.text)
                    detail_chunks.append(detail_chunk)
                if detail_chunks:
                    print(detail_chunks)
  #      detail_dict[count] = "|".join(sum(detail_chunks, []))
  #      count += 1
            
  #  print(detail_dict)
                    

In [272]:
nouns_and_modifiers(germinal_clauses[0:11])

[['the', 'open', 'plain']]
[['a', 'starless', 'sky']]
[['a', 'man']]
[['a', 'man', 'walked', 'alone', 'along', 'the', 'highway', 'from', 'marchiennes', 'to', 'montsou']]
[['the', 'highway']]
[['to', 'montsou']]
[['a', 'straight', 'paved', 'road', 'ten', 'kilometres', 'in', 'length']]
[['ten', 'kilometres', 'in', 'length']]
[['intersecting', 'the', 'beetroot', 'fields']]
[['the', 'beetroot', 'fields']]
[['he', 'could', 'not', 'even', 'see', 'the', 'black', 'soil', 'before', 'him']]
[['the', 'black', 'soil', 'before', 'him']]
[['only', 'felt', 'the', 'immense', 'flat', 'horizon', 'by', 'the', 'gusts', 'of', 'march', 'wind']]
[['the', 'immense', 'flat', 'horizon', 'by', 'the', 'gusts', 'of', 'march', 'wind']]
[['the', 'gusts', 'of', 'march', 'wind']]
[['march', 'wind']]
[['squalls', 'as', 'strong', 'as', 'on', 'the', 'sea']]
[['the', 'sea']]
[['and', 'frozen', 'from', 'sweeping', 'leagues', 'of', 'marsh', 'and', 'naked', 'earth']]
[['sweeping', 'leagues', 'of', 'marsh', 'and', 'naked', 'e